# Use text for stock regression price prediction

- https://www.kaggle.com/code/lseiyjg/use-news-to-predict-stock-markets

In [1]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

In [2]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = "SELECT title, content, date FROM crawl_data where domain = 'https://vneconomy.vn/kinh-te-the-gioi.htm'"
df = pd.read_sql(query, connection)

df['date_only'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')

from vnstock import *
from sklearn.model_selection import train_test_split

df_his = stock_historical_data('TPB', '2022-01-01', '2023-07-28', "1D", 'stock')
df_his['date'] = pd.to_datetime(df_his['time']).dt.strftime('%Y-%m-%d')


dfMerge = pd.merge(df, df_his, left_on=['date_only'], right_on=['date'], how='inner')
dfSumarize = dfMerge[['title', 'close', 'date_y']]

# Sorting the DataFrame by the 'date_column' in ascending order

df_sorted = dfSumarize.sort_values(by='date_y', ascending=True)
print(df_sorted.head())

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_66269/2159884431.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                  title  close      date_y
1257  Lo sợ lĩnh vực công nghiệp sụp đổ, châu Âu “vã...  14470  2022-10-07
1252  Lợi nhuận của Samsung giảm mạnh do nhu cầu chi...  14470  2022-10-07
1253  Giá vàng thế giới giảm, trong nước đồng loạt t...  14470  2022-10-07
1256      10 nhà sản xuất pin xe điện hàng đầu thế giới  14470  2022-10-07
1255  Chứng khoán Mỹ tiếp tục lao dốc, giá dầu giữ đ...  14470  2022-10-07


In [38]:
import sys
sys.path.insert(0, '../algorithm')
import regress
from importlib import reload
reload(regress)

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

modelClass = regress.PhobertTransform(phobert, tokenizer)

x_train_raw = df_sorted['title']
y_train_raw = df_sorted['close']

modelClass.fit(x_train_raw, y_train_raw)

# print(np.array(x_train).shape)

fit (1258,) (1258,)


/Users/ngocp/Documents/projects/pyml/stock/temp_testing/../algorithm/regress.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = self.model(torch.tensor(encodedTextInputIds))


fit_1: [14470 14470 14470 ... 18700 18700 18700]


In [39]:
test = [
    'Lạm phát khiến Hàn Quốc giảm ngân sách lần đầu',
]

# t = vectorize_mean(test)
t = modelClass.predict(test)
print(t)


[15694.28574872]


/Users/ngocp/Documents/projects/pyml/stock/temp_testing/../algorithm/regress.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = self.model(torch.tensor(encodedTextInputIds))


In [44]:
test = [
    'Lạm phát khiến Hàn Quốc giảm ngân sách lần đầu',
]

# t = vectorize_mean(test)
x_test_encoded = vectorize_mean_for(test)
print(len(x_test_encoded[0]), x_test_encoded)
result_pred = modelSvg.predict(x_test_encoded)
print(result_pred)

768 [array([-2.01829448e-02,  8.64872262e-02, -8.73286575e-02,  1.88893065e-01,
       -1.23140261e-01, -1.68129236e-01, -2.92146027e-01, -3.38564813e-01,
        2.12325752e-02,  1.26371041e-01,  8.35454166e-02,  5.97798750e-02,
        1.54909700e-01,  1.58262793e-02,  5.03614135e-02, -1.22325093e-01,
        2.16585770e-02,  2.43656784e-01, -6.98359534e-02,  2.20651329e-01,
       -8.83912891e-02,  7.27590695e-02, -1.29440024e-01, -3.01606767e-02,
       -3.38772833e-01,  3.08912486e-01,  1.22906407e-02,  3.96917872e-02,
       -2.48780832e-01, -1.77071065e-01, -2.27202073e-01,  8.64247903e-02,
        2.12022476e-02,  3.39687802e-02,  2.98975985e-02,  5.58604777e-01,
       -1.83685452e-01,  1.48079723e-01,  1.00964420e-01,  5.66273391e-01,
        1.44922987e-01,  1.14517875e-01,  1.71799272e-01,  1.07087240e-01,
        2.94297278e-01, -7.76214525e-02,  2.59682566e-01,  1.10911489e-01,
       -2.63571054e-01,  9.06947851e-02,  4.62090448e-02, -1.26314268e-01,
       -3.54175031e-

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_3844/3106091539.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = phobert(torch.tensor(encodedTextInputIds))


(10, 15, 768)

In [45]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Embedding, LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer

x_train = np.array(x_train)
input_dim = x_train.shape[1]
nb_classes = 1
# Here's a Deep Dumb MLP (DDMLP)
modelDL = Sequential()
modelDL.add(Dense(256, input_dim=input_dim))
modelDL.add(Activation('relu'))
modelDL.add(Dropout(0.4))
modelDL.add(Dense(128))
modelDL.add(Activation('relu'))
modelDL.add(Dropout(0.4))
modelDL.add(Dense(nb_classes))
modelDL.add(Activation('softmax'))
modelDL.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelDL.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               196864    
                                                                 
 activation_3 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [46]:
modelDL.fit(x_train, y_train)

42/42 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
